In [ ]:
import os
from pathlib import Path

import aiohttp

import datasets as ds

# タイムアウト値を秒単位で設定（例: 100秒）
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "36000"  # 👈 この行を追加

# 保存先ディレクトリを指定
local_dir = Path("datasets/JDocQA")
local_dir.mkdir(parents=True, exist_ok=True)

# 既にローカルに保存済みならそれを読み込む
if (local_dir / "dataset_info.json").exists():
    print("🔁 Loading dataset from local disk...")
    dataset = ds.load_from_disk(str(local_dir))
else:
    print("⬇️ Downloading dataset from Hugging Face Hub...")
    dataset = ds.load_dataset(
        path="shunk031/JDocQA",
        rename_pdf_category=True,
        trust_remote_code=True,
        storage_options={
            "client_kwargs": {"timeout": aiohttp.ClientTimeout(total=36000)}
        },
    )
    dataset.save_to_disk(str(local_dir))
    print(f"💾 Dataset saved locally to {local_dir}")

In [ ]:
from dotenv import load_dotenv

load_dotenv(verbose=True)

In [ ]:
import json
import os
import re
import textwrap
import time

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    AzureOpenAIEmbeddingSkill,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    FieldMapping,
    HnswAlgorithmConfiguration,
    IndexingParameters,
    InputFieldMappingEntry,
    LexicalAnalyzerName,
    OutputFieldMappingEntry,
    SearchField,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection,
    SearchIndexerDataSourceType,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    VectorSearch,
    VectorSearchProfile,
)
from azure.search.documents.models import QueryType
from azure.storage.blob import BlobServiceClient, ContentSettings
from tqdm import tqdm

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential, "https://search.azure.com/.default"
)

# どの split を使うか
SPLIT = "test"  # "validation" も可
MAX_SAMPLES = 300  # None で全件
CHUNK_SIZE = 700
CHUNK_OVERLAP = 200
USE_ORIGINAL = False

SEARCH_ENDPOINT = os.getenv("SEARCH_ENDPOINT")
AOAI_ENDPOINT = os.getenv("AOAI_ENDPOINT")
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AOAI_EMBEDDING_MODEL = "text-embedding-3-large"
AOAI_EMBEDDING_DEPLOYMENT = "text-embedding-3-large"
AOAI_GPT_MODEL = "gpt-5-mini"
AOAI_GPT_DEPLOYMENT = "gpt-5-mini"
INDEX_NAME = "jdocqa-index"
KNOWLEDGE_SOURCE_NAME = "jdocqa-knowledge-source"
KNOWLEDGE_AGENT_NAME = "jdocqa-knowledge-agent"
SEARCH_API_VERSION = "2025-08-01-preview"
DATA_SOURCE_NAME = "ds-jdocqa-chunks"
SKILLSET_NAME = "ss-jdocqa-embed"
INDEXER_NAME = "idx-jdocqa"
DIM = 3072
BLOB_CONTAINER = "jdocqa-chunks"
BLOB_PREFIX = "docs"

In [ ]:
# data = dataset[SPLIT]

# if MAX_SAMPLES is not None:
#     data = data.select(range(min(len(data), MAX_SAMPLES)))

# # 同一contextをユニーク化してdoc_idを割り当て
# contexts = [ex["context"] for ex in data]
# unique_contexts, inverse_indices = np.unique(contexts, return_inverse=True)
# # unique_contexts -> ["Xの本文", "Yの本文"]  ※辞書順
# # inverse_indices -> [0, 1, 0]              # 元の0番/2番はユニーク配列の0番に対応

# doc_ids = np.arange(len(unique_contexts))

# queries = [ex["question"] for ex in data]
# gold_doc_ids = inverse_indices


# def mk_doc(i: int) -> dict:
#     return {
#         "id": str(i),
#         "content": unique_contexts[i],
#         "pdf_category": str(data[i]["pdf_category"])
#         if i < len(data) and "pdf_category" in data[i]
#         else "N/A",
#     }


# docs = [mk_doc(i) for i in range(len(unique_contexts))]

# print(f"#docs={len(docs)}  #queries={len(queries)}  (split={SPLIT})")
# print("Example doc:", docs[155] if docs else None)
# print(
#     "Example QA:",
#     {"q": queries[0], "gold_doc_id": int(gold_doc_ids[0])} if queries else None,
# )

In [ ]:
def sentence_split_ja(text: str):
    t = re.sub(r"[\r\n]+", "。", text)
    t = re.sub(r"。+", "。", t)
    parts = [p.strip() for p in t.split("。") if p.strip()]
    return [p + "。" for p in parts]


def make_chunks(text: str, size=700, overlap=200):
    sents = sentence_split_ja(text)
    chunks, buf, cur = [], [], 0
    for s in sents:
        if cur + len(s) > size and buf:
            chunks.append("".join(buf))
            keep = "".join(buf)[-overlap:] if overlap > 0 else ""
            buf, cur = ([keep] if keep else []), len(keep)
        buf.append(s)
        cur += len(s)
    if buf:
        chunks.append("".join(buf))
    final = []
    for c in chunks:
        if len(c) <= size * 2:
            final.append(c)
        else:
            step = max(1, size - overlap)
            for i in range(0, len(c), step):
                final.append(c[i : i + size])
    return final


def stable_id(text: str) -> str:
    import hashlib

    return hashlib.sha256(text.encode("utf-8")).hexdigest()[:16]

In [ ]:
data = dataset[SPLIT]

if MAX_SAMPLES is not None:
    data = data.select(range(min(MAX_SAMPLES, len(data))))


def pick_ctx(ex):
    return (
        (ex.get("original_context") or ex.get("context"))
        if USE_ORIGINAL
        else ex.get("context")
    )


chunk_docs = {}
parent_to_chunks = {}
qa_gold_sets = []
queries = []

for ex in tqdm(data):
    ctx = pick_ctx(ex) or ""
    parent_id = stable_id(ctx) if ctx else stable_id("EMPTY")
    chs = make_chunks(ctx, CHUNK_SIZE, CHUNK_OVERLAP) if ctx else [" "]
    ids = []
    for ch in chs:
        cid = stable_id(parent_id + "|" + ch)
        if cid not in chunk_docs:
            chunk_docs[cid] = {
                "id": cid,
                "parent_id": parent_id,
                "content": ch,
                # "category": str(ex.get("category","unknown"))
            }
        ids.append(cid)
    parent_to_chunks[parent_id] = ids
    qa_gold_sets.append(set(ids))
    queries.append(ex.get("question", ""))

docs = list(chunk_docs.values())
print(f"parents={len(parent_to_chunks)} chunks={len(docs)} queries={len(queries)}")

In [ ]:
try:
    index_client.delete_index(INDEX_NAME)
except Exception:
    pass  # 存在しない場合は無視


index = SearchIndex(
    name=INDEX_NAME,
    fields=[
        SearchField(
            name="id",
            type="Edm.String",
            key=True,
            filterable=True,
        ),
        SearchField(
            name="parent_id",
            type="Edm.String",
            filterable=True,
        ),
        SearchField(
            name="content",
            type="Edm.String",
            searchable=True,
            analyzer_name=LexicalAnalyzerName.JA_LUCENE,
        ),
        SearchField(
            name="page_embedding_text_3_large",
            type="Collection(Edm.Single)",
            stored=True,
            retrievable=True,
            vector_search_dimensions=DIM,
            vector_search_profile_name="hnsw_text_3_large",
        ),
        # SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        #
        # SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[
            VectorSearchProfile(
                name="hnsw_text_3_large",
                algorithm_configuration_name="alg",
                vectorizer_name="azure_openai_text_3_large",
            )
        ],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=AOAI_ENDPOINT,
                    deployment_name=AOAI_EMBEDDING_DEPLOYMENT,
                    model_name=AOAI_EMBEDDING_MODEL,
                ),
            )
        ],
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[SemanticField(field_name="content")]
                ),
            )
        ],
    ),
)

index_client = SearchIndexClient(endpoint=SEARCH_ENDPOINT, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{INDEX_NAME}' created or updated successfully.")

In [ ]:
# print(f"Uploading {len(docs)} documents to index '{index_name}'...")
# print(f"Sample document: {docs[0] if docs else 'N/A'}")

# with SearchIndexingBufferedSender(endpoint=SEARCH_ENDPOINT, index_name=index_name, credential=credential) as client:
#     client.upload_documents(documents=docs)

# print(f"Documents uploaded to index '{index_name}' successfully.")

In [ ]:
bsc = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
container_client = bsc.get_container_client(BLOB_CONTAINER)
try:
    container_client.create_container()
except Exception:
    pass


def upload_docs_to_blob(docs, prefix=BLOB_PREFIX):
    for d in tqdm(docs):
        name = f"{prefix}/{d['id']}.json"
        data = json.dumps(d, ensure_ascii=False).encode("utf-8")
        container_client.upload_blob(
            name,
            data,
            overwrite=True,
            content_settings=ContentSettings(content_type="application/json"),
        )
    return len(docs)


print("Uploaded:", upload_docs_to_blob(docs))

In [ ]:
container = SearchIndexerDataContainer(
    name=BLOB_CONTAINER,
    query=BLOB_PREFIX,  # "docs" 以下だけ取り込む。コンテナ全体なら None
)

data_source = SearchIndexerDataSourceConnection(
    name=DATA_SOURCE_NAME,
    type=SearchIndexerDataSourceType.AZURE_BLOB,  # 他: AZURE_TABLE, AZURE_SQL, COSMOSDB, ADLSGEN2 など
    connection_string=AZURE_STORAGE_CONNECTION_STRING,
    container=container,
    description="JDocQA chunk JSONs in Blob Storage",
)

# 既存があれば更新、なければ作成
indexer_client = SearchIndexerClient(endpoint=SEARCH_ENDPOINT, credential=credential)
indexer_client.create_or_update_data_source_connection(data_source)

In [ ]:
for source in indexer_client.get_data_source_connections():
    print(source.name, source.type)

print(f"Data Source '{DATA_SOURCE_NAME}' created or updated.")

In [ ]:
try:
    indexer_client.delete_skillset(SKILLSET_NAME)
except Exception:
    pass  # 存在しない場合は無視

# ===== Skillset 作成 =====
# AzureOpenAI Embedding Skill を "汎用スキル" として作成し、追加プロパティで必要項目を渡す
embedding_skill = AzureOpenAIEmbeddingSkill(
    description="Skill to generate embeddings via Azure OpenAI",
    context="/document",
    resource_url=AOAI_ENDPOINT,
    deployment_name=AOAI_EMBEDDING_DEPLOYMENT,
    model_name=AOAI_EMBEDDING_MODEL,
    dimensions=DIM,
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/content"),
    ],
    outputs=[OutputFieldMappingEntry(name="embedding", target_name="emb")],
)

skillset = SearchIndexerSkillset(
    name=SKILLSET_NAME,
    skills=[embedding_skill],
    description="JDocQA index-time embedding skillset",
)

# 既存があれば更新、なければ作成
indexer_client.create_or_update_skillset(skillset)

print("Skillset created.")

In [ ]:
try:
    indexer_client.delete_indexer(INDEXER_NAME)
    print(f"Deleted indexer: {INDEXER_NAME}")
except HttpResponseError as e:
    # 404 相当は無視（存在しない場合）
    if getattr(e, "status_code", None) not in (404,):
        raise

indexer = SearchIndexer(
    name=INDEXER_NAME,
    data_source_name=DATA_SOURCE_NAME,
    target_index_name=INDEX_NAME,
    skillset_name=SKILLSET_NAME,  # スキルセットを紐づけ
    # ドキュメント -> インデックス のフィールドマッピング
    field_mappings=[
        FieldMapping(source_field_name="id", target_field_name="id"),
        FieldMapping(source_field_name="parent_id", target_field_name="parent_id"),
        FieldMapping(source_field_name="content", target_field_name="content"),
    ],
    # スキル出力 -> インデックス のフィールドマッピング
    # 例：EmbeddingSkill の出力 "/document/emb" を vector フィールド "content_vector" へ
    output_field_mappings=[
        FieldMapping(
            source_field_name="/document/emb",
            target_field_name="page_embedding_text_3_large",
        ),
    ],
    # インデクサのパラメータ
    parameters=IndexingParameters(
        configuration={
            "parsingMode": "json",  # 1 JSON = 1 ドキュメント
            "failOnUnsupportedContentType": False,  # 未対応MIMEで失敗させない
        }
    ),
)

indexer_client.create_or_update_indexer(indexer)

In [ ]:
# 1) Indexer を手動実行
try:
    indexer_client.reset_indexer(
        INDEXER_NAME
    )  # 1) 変更追跡（ハイウォーターマーク）をリセット
    indexer_client.run_indexer(INDEXER_NAME)
    print(f"Run requested: {INDEXER_NAME}")
except HttpResponseError as e:
    print("Run failed:", e)
    raise

# 2) 簡易ポーリング（状態が terminal になるまで待つ）
terminal = {"success", "transientFailure", "persistentFailure", "reset"}
for i in range(60):  # 最大 ~5分待機（5秒×60）
    st = indexer_client.get_indexer_status(INDEXER_NAME)
    last = st.last_result
    status = getattr(last, "status", None)
    processed = getattr(last, "items_processed", None)
    failed = getattr(last, "items_failed", None)
    print(f"[{i}] status={status} processed={processed} failed={failed}")

    if status in terminal:
        break
    time.sleep(5)

# 3) 結果チェック
if status != "success":
    raise RuntimeError(
        f"Indexer did not succeed. status={status}, processed={processed}, failed={failed}"
    )
print("Indexer run completed successfully.")

In [ ]:
search_client = SearchClient(
    endpoint=SEARCH_ENDPOINT,
    index_name=INDEX_NAME,
    credential=credential,
    api_version=SEARCH_API_VERSION,
)

In [ ]:
query = "家族手当の支給条件は？"

results = search_client.search(
    search_text=query,
    query_type=QueryType.SIMPLE,  # または QueryType.SEMANTIC（semantic設定がある場合）
    top=5,
    select=["id", "parent_id", "content", "page_embedding_text_3_large"],
)

for r in results:
    print(
        r["id"],
        r["@search.score"],
        r["content"][:80],
        r["page_embedding_text_3_large"][:5],
    )

In [ ]:
vq = VectorizableTextQuery(
    text="家族手当の支給条件は？",
    k_nearest_neighbors=5,
    fields="page_embedding_text_3_large",
)

results = search_client.search(
    search_text=None, vector_queries=[vq], select=["id", "parent_id", "content"]
)

for r in results:
    print(r["id"], r["@search.score"], r["content"][:80])

In [ ]:
query = "家族手当の支給条件は？"

vector_query = VectorizableTextQuery(
    text=query, k_nearest_neighbors=50, fields="page_embedding_text_3_large"
)

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["id", "parent_id", "content"],
    top=5,
)

for r in results:
    print(r["id"], r["@search.score"], r["content"][:80])

In [ ]:
ks = SearchIndexKnowledgeSource(
    name=KNOWLEDGE_SOURCE_NAME,
    description="Knowledge source for Earth at night data",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=INDEX_NAME,
        source_data_select="id,content,page_embedding_text_3_large",
    ),
)

index_client.create_or_update_knowledge_source(
    knowledge_source=ks, api_version=SEARCH_API_VERSION
)
print(f"Knowledge source '{KNOWLEDGE_SOURCE_NAME}' created or updated successfully.")

In [ ]:
aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=AOAI_ENDPOINT,
    deployment_name=AOAI_GPT_DEPLOYMENT,
    model_name=AOAI_GPT_MODEL,
)

output_cfg = KnowledgeAgentOutputConfiguration(
    modality=KnowledgeAgentOutputConfigurationModality.ANSWER_SYNTHESIS,
    include_activity=True,
)

agent = KnowledgeAgent(
    name=KNOWLEDGE_AGENT_NAME,
    models=[KnowledgeAgentAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(
            name=KNOWLEDGE_SOURCE_NAME,
            reranker_threshold=2.5,
        )
    ],
    output_configuration=output_cfg,
)

index_client.create_or_update_agent(agent, api_version=SEARCH_API_VERSION)
print(f"Knowledge agent '{KNOWLEDGE_AGENT_NAME}' created or updated successfully.")

In [ ]:
instructions = """
あなたは JDocQA（日本語ドキュメントQA）の回答エージェントです。
与えられた「検索済みチャンク（content, id, parent_id, page_number, category などのメタデータ付き）」のみを根拠に日本語で回答してください。
根拠が足りない場合や結論できない場合は、正確に「I don't know」と答えます。

【回答原則】
1) 出典制約: 回答は常に与えられたチャンクの内容に限定。外部知識や推測で補わない。
2) 粒度: 質問の粒度に合わせ、過不足のない要約で答える。具体値や定義はチャンクにあるもののみ使用。
3) 矛盾対応: チャンク間で内容が食い違う場合は矛盾を明示し、より具体・新しい・法令/規程ベースのものを優先。
4) 不確実性: 情報が不十分/曖昧なら「I don't know」または条件つき回答（根拠を示す）。
5) 引用: 可能な限り根拠チャンクの ID を `[doc:{id}]` 形式で末尾に添える（複数可）。
6) 言語: 出力は自然な日本語。箇条書きは最大3–6項目に抑える。

【作業手順（内部方針）】
- 質問を分解し、回答に必要な事実項目を列挙。
- 提供チャンクから該当箇所を抽出し、重複・矛盾・前後関係を整理。
- 必要最小限の要約・言い換えを行い、日本語で簡潔に記述。
- 該当チャンクの `id` を引用として付与（例: [doc:0053c7...]）。
- 十分な根拠がなければ「I don't know」。

【禁止事項】
- 断定のための補完推測、年代/数値の創作、URLや図表の創作。
- JDocQAに含まれない範囲へ話を広げること。

【回答フォーマット例】
- 単文回答: 「〜です。[doc:abc123]」
- 箇条書き: 複数条件/手順があるときのみ使用。
- 不明: 「I don't know」
"""


instructions_formatted = """
You are a Japanese RAG answerer for JDocQA. Use ONLY the provided chunks.
If insufficient evidence: reply with JSON whose "answer" is exactly "I don't know".

Output MUST be a single JSON object:
{
  "answer": string,            // Japanese final answer or "I don't know"
  "citations": [ { "id": string } ],  // chunk ids you used (at least one if answer != "I don't know")
  "confidence": "high" | "medium" | "low"
}

Rules:
- No outside knowledge. No speculation.
- Keep it brief and specific. If chunks conflict, note it briefly and choose the most specific source.
- If you used multiple chunks, include all their ids in citations.
"""

messages = [{"role": "system", "content": instructions}]

In [ ]:
agent_client = KnowledgeAgentRetrievalClient(
    endpoint=SEARCH_ENDPOINT, agent_name=KNOWLEDGE_AGENT_NAME, credential=credential
)
query_1 = """
    家族手当の支給条件は？
    """

messages.append({"role": "user", "content": query_1})

req = KnowledgeAgentRetrievalRequest(
    messages=[
        KnowledgeAgentMessage(
            role=m["role"],
            content=[KnowledgeAgentMessageTextContent(text=m["content"])],
        )
        for m in messages
        if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=KNOWLEDGE_SOURCE_NAME, kind="searchIndex"
        )
    ],
)

result = agent_client.retrieve(retrieval_request=req, api_version=SEARCH_API_VERSION)
print(f"Retrieved content from '{KNOWLEDGE_SOURCE_NAME}' successfully.")

In [ ]:
print("Response")
print(textwrap.fill(result.response[0].content[0].text, width=120))

print("Activity")
print(json.dumps([a.as_dict() for a in result.activity], indent=2, ensure_ascii=False))

print("Results")
print(
    json.dumps([r.as_dict() for r in result.references], indent=2, ensure_ascii=False)
)